In [ ]:
from google.colab import drive

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test.csv')

In [3]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

In [4]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [5]:
#delay 1, 0 으로 변환
dic = {'Not_Delayed' : 0, 'Delayed' : 1}
train_df['Delay'] = train_df['Delay'].map(dic) # delay float타입으로 변경

In [6]:
train_df.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN


주를 집어넣으면 dict로 생성해서 시차를 계산해줌.

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = train_df[train_df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        time_delay_dict[state, st] = -((time_delay/2)//60)*60
    
    return time_delay_dict

모든 주에 대해 출력!!

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 60.0, ('Oklahoma', 'Colorado'): -60.0, ('Oklahoma', nan): nan, ('Oklahoma', 'Georgia'): 120.0, ('Oklahoma', 'Illinois'): 60.0, ('Oklahoma', 'New Jersey'): 120.0, ('Oklahoma', 'California'): -120.0, ('Oklahoma', 'Virginia'): 120.0, ('Oklahoma', 'Florida'): 120.0, ('Oklahoma', 'Missouri'): -0.0, ('Oklahoma', 'Nevada'): -0.0, ('Oklahoma', 'North Carolina'): 120.0, ('Oklahoma', 'Minnesota'): 60.0, ('Oklahoma', 'Arizona'): -60.0, ('Oklahoma', 'Utah'): -60.0, ('Oklahoma', 'Maryland'): 120.0, ('Oklahoma', 'Michigan'): 60.0, ('Oklahoma', 'Pennsylvania'): 60.0, ('Oklahoma', 'Washington'): -120.0, ('Oklahoma', 'Tennessee'): nan, ('Illinois', 'Utah'): -60.0, ('Illinois', 'Missouri'): -0.0, ('Illinois', 'Kentucky'): 60.0, ('Illinois', 'Tennessee'): -0.0, ('Illinois', 'Ohio'): 60.0, ('Illinois', 'Minnesota'): -0.0, ('Illinois', 'West Virginia'): 120.0, ('Illinois', 'Michigan'): 60.0, ('Illinois', 'Oklahoma'): -0.0, ('Illinois', 'Washington'): -0.0, ('Illinois', 'New York'): 

**문제점**

('Kentucky', 'Washington'): nan 이런식의 문제점이 발생한다.

In [ ]:
s = pd.DataFrame(time_delay, index=['time_delay']).T

In [ ]:
s

time_delay
Oklahoma                                       Texas                                                 60.0
                                               Colorado                                             -60.0
                                               NaN                                                    NaN
                                               Georgia                                              120.0
                                               Illinois                                              60.0
...                                                                                                   ...
Rhode Island                                   Texas                                                -60.0
                                               Louisiana                                              NaN
U.S. Pacific Trust Territories and Possessions U.S. Pacific Trust Territories and Possessions        -0.0
                                               Hawaii                                               120.0
                                               NaN                                                    NaN

[1398 rows x 1 columns]

nan인 주들 이미 있는 주들 만으로도 시차계산이 가능할듯하다.

밑에 사진은 관측할 수 없는 주들이다.


해결책

일단 nan인 것은 제외하고 생각하자.

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = train_df[train_df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 0, ('Oklahoma', 'Colorado'): -2, ('Oklahoma', 'Georgia'): 1, ('Oklahoma', 'Illinois'): 0, ('Oklahoma', 'New Jersey'): 1, ('Oklahoma', 'California'): -3, ('Oklahoma', 'Virginia'): 1, ('Oklahoma', 'Florida'): 1, ('Oklahoma', 'Missouri'): 0, ('Oklahoma', 'Nevada'): -1, ('Oklahoma', 'North Carolina'): 2, ('Oklahoma', 'Minnesota'): 0, ('Oklahoma', 'Arizona'): -2, ('Oklahoma', 'Utah'): -2, ('Oklahoma', 'Maryland'): 1, ('Oklahoma', 'Michigan'): 1, ('Oklahoma', 'Pennsylvania'): 1, ('Oklahoma', 'Washington'): -3, ('Illinois', 'Utah'): -1, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 0, ('Illinois', 'Tennessee'): 0, ('Illinois', 'Ohio'): 0, ('Illinois', 'Minnesota'): 0, ('Illinois', 'West Virginia'): 2, ('Illinois', 'Michigan'): 1, ('Illinois', 'Oklahoma'): 0, ('Illinois', 'Washington'): 0, ('Illinois', 'New York'): 1, ('Illinois', 'Texas'): 0, ('Illinois', 'California'): 0, ('Illinois', 'Indiana'): 1, ('Illinois', 'Illinois'): 0, ('Illinois', 'Pennsylvania'): 0

내 풀이 

In [11]:
df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Distance','Delay']]

In [12]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,NaN,NaN,678.0,NaN
999996,TRAIN_999996,920.0,1028.0,NaN,Minnesota,223.0,NaN
999997,TRAIN_999997,800.0,1340.0,NaN,Texas,1642.0,NaN
999998,TRAIN_999998,1613.0,1824.0,Tennessee,NaN,214.0,NaN


In [13]:
df = df.dropna(subset=['Origin_State','Destination_State']) #결측치 제거 

In [22]:
df.isna().sum()

ID                               0
Estimated_Departure_Time     86628
Estimated_Arrival_Time       86516
Origin_State                     0
Destination_State                0
Distance                         0
Delay                       591429
dtype: int64

In [ ]:
state_list = df['Origin_State'].unique().tolist()
state_list

In [ ]:
list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist())) #Alabama를 입력해주는 값에 따라 달라지기 ~

['Illinois',
 'North Carolina',
 'Georgia',
 'Florida',
 'Texas',
 'New York',
 'Virginia',
 'Nevada',
 'Michigan',
 'Pennsylvania',
 'Maryland',
 'Colorado']

In [ ]:
#과정
# 1 . orinin_state == a / a는 전체 52개 state중 하나 
# 2. a에서 출발/ 도착한 state b -- b는 list(set(df[df['Origin_State']=='a']['Destination_State'].values.tolist())) 이 리스트에서 하나 
# 3 . 출발주 a / 도착주 b -- 여기서 도착시간 - 출발시간 의 평균 --  첫번째 시간 A_time  
# 4. 출발 b / 도착 a -- 여기서 도착시간- 출발시간의 평균 --  두번째 시간  B_time
# 5 . (A_time - B_time)/2 

In [ ]:
#1. origin_state ==a 
df[df['Origin_State']=='Alabama']

#2. a(Alabama)에서 출발하고 도착한 주 리스트로 정리
destination_list = list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist()))

#3. 출발주a / 도착주 b-- 위에 정리한 리스트 중 하나 
for b in destination_list:
    df[(df['Origin_State']=='Alabama') & (df['Destination_State']==b)]

df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Illinois')]
df[(df['Origin_State']=='Illinois') & (df['Destination_State']=='Alabama')]

In [ ]:
destination_list = list(set(df[df['Origin_State']=='Alabama']['Destination_State'].values.tolist()))
destination_list #state 출발 시 도착 state 리스트 

In [ ]:
a = df[(df['Origin_State']=='Alabama') & (df['Destination_State']=='Illinois')]
a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()  #1번 

In [ ]:
b = df[(df['Origin_State']=='Illinois') & (df['Destination_State']=='Alabama')]
b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()  #2번 
# b.isna().sum()

In [ ]:
#최종 
# 1번 - 2번 /2 
(a_time - b_time)/2

9.706989529712502

In [ ]:
destination_list #state = ;Alibama

['Illinois',
 'North Carolina',
 'Georgia',
 'Florida',
 'Texas',
 'New York',
 'Virginia',
 'Nevada',
 'Michigan',
 'Pennsylvania',
 'Maryland',
 'Colorado']

In [ ]:
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st] #출발a , 도착[1,2,3,4]  for문 
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)

내거 

In [26]:
def time_diff(df,state):
    destination_list =  list(set(df[df['Origin_State']==state]['Destination_State'].values.tolist()))
    time_diff_dict = {}
    for d in destination_list:
        a = df[(df['Origin_State']==state) & (df['Destination_State']==d)] #ali 출발 , texas 도착
        b = df[(df['Origin_State']==d) & (df['Destination_State']==state)] #texas 출. ali 도착
        a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
        b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
        time_di = (a_time - b_time)/2 #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
        time_di = round(time_di)
        time_diff_dict[state, d] = time_di
    
    return time_diff_dict

In [ ]:
time_delay = dict() #시간 결측치가 존재하는 데이터의 경우 오류발생
for i in state_list:
    time_delay.update(time_diff(df, i))
print(time_delay)

In [29]:
destination_list =  list(set(df[df['Origin_State']=='Oklahoma']['Destination_State'].values.tolist()))
destination_list

['Nevada',
 'Minnesota',
 'Tennessee',
 'Pennsylvania',
 'Virginia',
 'Missouri',
 'California',
 'North Carolina',
 'Utah',
 'Illinois',
 'Michigan',
 'New Jersey',
 'Colorado',
 'Maryland',
 'Arizona',
 'Georgia',
 'Florida',
 'Washington',
 'Texas']

In [35]:
a = df[(df['Origin_State']=='Oklahoma') & (df['Destination_State']=='Texas')] #ali 출발 , texas 도착
b = df[(df['Origin_State']=='Texas') & (df['Destination_State']=='Oklahoma')] #texas 출. ali 도착
a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
time_di = (a_time - b_time)/2 #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
time_di = round(time_di)
# time_diff_dict[state, d] = time_di
time_di

5

In [30]:
df[df['Origin_State']=='Oklahoma']

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
510,TRAIN_000510,1818.0,NaN,Oklahoma,Texas,140.0,NaN
827,TRAIN_000827,550.0,715.0,Oklahoma,Texas,419.0,NaN
1031,TRAIN_001031,1408.0,1540.0,Oklahoma,Texas,429.0,NaN
1365,TRAIN_001365,1301.0,1340.0,Oklahoma,Colorado,495.0,1.0
...,...,...,...,...,...,...,...
998918,TRAIN_998918,1813.0,NaN,Oklahoma,Texas,175.0,NaN
998970,TRAIN_998970,1854.0,2158.0,Oklahoma,Georgia,761.0,0.0
999380,TRAIN_999380,1325.0,1412.0,Oklahoma,Colorado,541.0,NaN
999503,TRAIN_999503,1955.0,2040.0,Oklahoma,Nevada,986.0,NaN


In [28]:
time_diff(df, 'Oklahoma')

ValueError: ignored

In [25]:
time_delay = dict() #시간 결측치가 존재하는 데이터의 경우 오류발생
for i in state_list:
    if not np.isnan(time_delay):
        time_delay.update(time_diff(df, i))
print(time_delay)

TypeError: ignored

In [20]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999990,TRAIN_999990,NaN,751.0,Illinois,Texas,802.0,NaN
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0,NaN
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0,0.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0,NaN


In [ ]:
ex #여기서  money 라는 컬럼에 gdp x gdp rate를 해보기 

In [ ]:
import pandas as pd
import numpy as np
animal = pd.DataFrame(np.array([['사자','1','10','어흥','010-1234-1234'],
                                ['사자','2','20','야옹','010-1234-4321'],
                                ['늑대','1','30','컹컹','010-4331-1234'],
                                ['하마','2','40','므허','010-1234-3567'],
                                ['돼지','3','10','꿀꿀','010-1234-9876']]),
                      columns = ['종류','반','번호','별명','연락처'])

In [ ]:
animal

,종류,반,번호,별명,연락처
0,사자,1,10,어흥,010-1234-1234
1,사자,2,20,야옹,010-1234-4321
2,늑대,1,30,컹컹,010-4331-1234
3,하마,2,40,므허,010-1234-3567
4,돼지,3,10,꿀꿀,010-1234-9876


In [ ]:
animal['a'] = animal.apply(lambda  x : x['번호'] if x['종류']=='사자', axis=1)


In [ ]:
animal

,종류,반,번호,별명,연락처,반-번호,a
0,사자,1,10,어흥,010-1234-1234,자퇴,10
1,사자,2,20,야옹,010-1234-4321,2-20,20
2,늑대,1,30,컹컹,010-4331-1234,1-30,1
3,하마,2,40,므허,010-1234-3567,2-40,2
4,돼지,3,10,꿀꿀,010-1234-9876,3-10,3


In [ ]:
animal['반-번호'] = animal.apply(lambda x:x['반']+'-'+x['번호'], axis = 1)

In [ ]:
animal['반-번호'] = animal.apply(lambda x:'자퇴' if x['별명']=='어흥' else x['반']+'-'+x['번호'], axis=1)

In [ ]:
#시차 까지 구했는데 이 시차를 데이터 프레임 안에 time_diff 컬럼에 새로 넣어야함
# for 문 한번 돌때ㅔ 마다 . state와 d (destination_list 원소 하나)의 시차를 구하게 되는데 이 시차를, 출발state도착d & 출발d도착state 인 데이터에 넣어야함


In [ ]:
time_diff('Alabama')

9.706989529712502

In [ ]:
data = df.head()
data

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN


In [ ]:
data['diff_time'] = data.apply(lambda x : x['Estimation_Departure_Time'] )

In [ ]:
#1 ) 비행시간 > 시차계산 > 시차를 계산한 비행시간 ex) 도착-출발=3시간, 시차 1시간, 비행시간 = 4시간 > 거리별 비행시간을 계산하여 결측치 채워넣기  /// 여기서 이상치 계산하기  
#2 ) 

baseline code

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [36]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test.csv')

In [37]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

In [38]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [39]:
#delay 1, 0 으로 변환
dic = {'Not_Delayed' : 0, 'Delayed' : 1}
train_df['Delay'] = train_df['Delay'].map(dic) # delay float타입으로 변경

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = train_df[train_df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        time_delay_dict[state, st] = -((time_delay/2)//60)*60
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 60.0, ('Oklahoma', 'Colorado'): -60.0, ('Oklahoma', nan): nan, ('Oklahoma', 'Georgia'): 120.0, ('Oklahoma', 'Illinois'): 60.0, ('Oklahoma', 'New Jersey'): 120.0, ('Oklahoma', 'California'): -120.0, ('Oklahoma', 'Virginia'): 120.0, ('Oklahoma', 'Florida'): 120.0, ('Oklahoma', 'Missouri'): -0.0, ('Oklahoma', 'Nevada'): -0.0, ('Oklahoma', 'North Carolina'): 120.0, ('Oklahoma', 'Minnesota'): 60.0, ('Oklahoma', 'Arizona'): -60.0, ('Oklahoma', 'Utah'): -60.0, ('Oklahoma', 'Maryland'): 120.0, ('Oklahoma', 'Michigan'): 60.0, ('Oklahoma', 'Pennsylvania'): 60.0, ('Oklahoma', 'Washington'): -120.0, ('Oklahoma', 'Tennessee'): nan, ('Illinois', 'Utah'): -60.0, ('Illinois', 'Missouri'): -0.0, ('Illinois', 'Kentucky'): 60.0, ('Illinois', 'Tennessee'): -0.0, ('Illinois', 'Ohio'): 60.0, ('Illinois', 'Minnesota'): -0.0, ('Illinois', 'West Virginia'): 120.0, ('Illinois', 'Michigan'): 60.0, ('Illinois', 'Oklahoma'): -0.0, ('Illinois', 'Washington'): -0.0, ('Illinois', 'New York'): 

In [ ]:
s = pd.DataFrame(time_delay, index=['time_delay']).T

In [51]:
def find_time_lag(df, state):
    OA = df[df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = df[df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        # if train_df = train_df[train_df['Estimated_Arrival_Time'] > train_df['Estimated_Departure_Time']]: #도착시간 > 출발시간 
        st_oa = OA[OA['Destination_State'] == st]
        st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = (st_da.Estimated_Arrival_Time - st_da.Estimated_Departure_Time).mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

In [52]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 0, ('Oklahoma', 'Colorado'): -2, ('Oklahoma', 'Georgia'): 1, ('Oklahoma', 'Illinois'): 0, ('Oklahoma', 'New Jersey'): 1, ('Oklahoma', 'California'): -3, ('Oklahoma', 'Virginia'): 1, ('Oklahoma', 'Florida'): 1, ('Oklahoma', 'Missouri'): 0, ('Oklahoma', 'Nevada'): 0, ('Oklahoma', 'North Carolina'): 2, ('Oklahoma', 'Minnesota'): 0, ('Oklahoma', 'Arizona'): -2, ('Oklahoma', 'Utah'): -1, ('Oklahoma', 'Maryland'): 1, ('Oklahoma', 'Michigan'): 1, ('Oklahoma', 'Pennsylvania'): 1, ('Oklahoma', 'Washington'): -3, ('Illinois', 'Utah'): -1, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 0, ('Illinois', 'Tennessee'): 0, ('Illinois', 'Ohio'): 0, ('Illinois', 'Minnesota'): 0, ('Illinois', 'West Virginia'): 2, ('Illinois', 'Michigan'): 1, ('Illinois', 'Oklahoma'): 0, ('Illinois', 'Washington'): 0, ('Illinois', 'New York'): 0, ('Illinois', 'Texas'): 0, ('Illinois', 'California'): 0, ('Illinois', 'Indiana'): 1, ('Illinois', 'Illinois'): 0, ('Illinois', 'Pennsylvania'): 0,

In [53]:
time_delay['Oklahoma', 'Texas']

0

In [ ]:
OA = df[df['Origin_State'] == 'Oklahoma'] #출발지가 오클라호마
predict_st = OA.Destination_State.unique().tolist() #츌벌자거 오클라호마인 비행편의 도착지 리스트
DA = df[df['Destination_State'] == 'Oklahoma'] #도착지가 오클라호마
time_delay_dict = {}
st_oa = OA[OA['Destination_State'] == 'Texas'] #출발지가 오클라호마이고 도착지가 텍사스
st_da = DA[DA['Origin_State'] == 'Texas'] #출발지 텍사스 도착지 오클라호마
# if st_da['Estimated_Arrival_Time'].isna()  -->  데이터에 결측치가 존재하지 않을 때만 계산 
if st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']: #출발지가 오클라호마이고 도착지가텍사스일때 도착시간이 출발시간 보다 클때. /
    st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean() #출발지가 오클라호마이고 도착지가 텍사스인 비행편의 도착시간- 출발시간 의 평균 --> 도착시간이 출발시간보다 큰 것만 계산하기
if st_da['Estimated_Arrival_Time'] > st_da['Estimated_Departure_Time']: #출발지텍사스,도착지 오클라호마일 때 도착시간이 출발시간보다 클때 
    st_da_diff = (st_da.Estimated_Arrival_Time - st_da.Estimated_Departure_Time).mean()
time_delay = st_da_diff - st_oa_diff

위와 같이 할 때 문제점. 도착시간 혹은 출발시간에 결측치가 존재하기 때문에   
if 문의 시간 비교가 힘듬. 그렇다면 데이터가 존재하는 는 경우에만 비교하면 되지 않을까 

In [98]:
OA = df[df['Origin_State'] == 'Oklahoma'] #출발지가 오클라호마
predict_st = OA.Destination_State.unique().tolist() #츌벌자거 오클라호마인 비행편의 도착지 리스트
DA = df[df['Destination_State'] == 'Oklahoma'] #도착지가 오클라호마
time_delay_dict = {}
st_oa = OA[OA['Destination_State'] == 'Texas'] #출발지가 오클라호마이고 도착지가 텍사스
st_da = DA[DA['Origin_State'] == 'Texas']
st_oa = st_oa[st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']]
st_da = st_da[st_da['Estimated_Arrival_Time'] > st_da['Estimated_Departure_Time']]
st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean() #출발지가 오클라호마이고 도착지가 텍사스인 비행편의 도착시간- 출발시간 의 평균 --> 도착시간이 출발시간보다 큰 것만 계산하기
st_da_diff = (st_da.Estimated_Arrival_Time - st_da.Estimated_Departure_Time).mean()
time_delay = st_da_diff - st_oa_diff
time_delay

-4.349988799443324

In [94]:
OA = df[df['Origin_State'] == 'Oklahoma'] #출발지가 오클라호마
predict_st = OA.Destination_State.unique().tolist() #츌벌자거 오클라호마인 비행편의 도착지 리스트
DA = df[df['Destination_State'] == 'Oklahoma'] #도착지가 오클라호마
time_delay_dict = {}
st_oa = OA[OA['Destination_State'] == 'Texas'] #출발지가 오클라호마이고 도착지가 텍사스
st_da = DA[DA['Origin_State'] == 'Texas']

In [ ]:
st_oa = st_oa[st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']]
st_oa

In [ ]:
st_da = st_da[st_da['Estimated_Arrival_Time'] > st_da['Estimated_Departure_Time']]
st_da

In [ ]:
st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean() #출발지가 오클라호마이고 도착지가 텍사스인 비행편의 도착시간- 출발시간 의 평균 --> 도착시간이 출발시간보다 큰 것만 계산하기
st_oa_diff

In [ ]:
st_oa['Estimated_Arrival_Time']

In [ ]:
st_oa = OA[OA['Destination_State'] == 'Texas']
st_oa

In [ ]:
st_oa[st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']]


In [ ]:
train_df = train_df[train_df['Estimated_Arrival_Time'] > train_df['Estimated_Departure_Time']]

In [ ]:
s = pd.DataFrame(time_delay, index=['time_delay']).T
s.unstack().head()